# FINAL PROJECT: ÍÑIGO BARCELÓ ÁLVAREZ / JUAN LEAL ALIAGA

## 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data or newtype) for each of them. 

In [1]:
data Date = Date 
  { day   :: Int
  , month :: String
  , year  :: Int
  } deriving (Show, Eq, Ord)

data Person = Person 
  { firstName :: String
  , lastName  :: String
  , birthDate :: Date
  } deriving (Show, Eq)

data Publication
  = Book
      { bookId    :: String       -- Unique signature
      , author    :: Person       -- Single author
      , title     :: String
      , pages     :: Int
      , copies    :: [Copy]       -- List of copies
      }
  | Journal
      { journalId :: String       -- Unique signature
      , authors   :: [Person]     -- Multiple authors
      , title     :: String
      , releaseDate :: Date
      , pages     :: Int
      , copies    :: [Copy]
      }
  | DVD
      { dvdId     :: String       -- Unique signature
      , director  :: Person       -- Director is a single person
      , actors    :: [Person]     -- List of actors
      , title     :: String
      , releaseDate :: Date
      , duration  :: Int          -- Duration in minutes
      , copies    :: [Copy]
      }
  deriving (Show, Eq)

data Copy = Copy
  { isBorrowed  :: Bool
  , borrower    :: Maybe User    -- User who borrowed the copy (if any)
  , loanDate    :: Maybe Date    -- Loan date (if borrowed)
  , returnDate  :: Maybe Date    -- Expected return date (if borrowed)
  } deriving (Show, Eq)

data User
  = Student 
      { studentName :: Person
      , borrowed    :: [Copy]    -- Currently borrowed copies (¿MAX?)
      }
  | Professor
      { professorName :: Person
      , borrowed      :: [Copy] -- ¿MAX?
      }
  
  deriving (Show, Eq)

type Catalog = [Publication]

## 2. Functions to show the former concepts in a pretty way. 

In [2]:
instance Show Date where
  show (Date d m y) = show d ++ " " ++ m ++ " " ++ show y
  
instance Show Person where
  show (Person fName lName bDate) = 
    fName ++ " " ++ lName ++ " (" ++ show bDate ++ ")"
    
instance Show Copy where
  show (Copy isB mbUser mbLoanDate mbReturnDate)
    | isB = "Borrowed by " ++ maybe "Unknown" show mbUser ++
            " | Loan Date: " ++ maybe "N/A" show mbLoanDate ++
            " | Return Date: " ++ maybe "N/A" show mbReturnDate
    | otherwise = "Available"
    
instance Show Publication where
  show (Book id author title pages copies) =
    "Book [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Author: " ++ show author ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (Journal id authors title releaseDate pages copies) =
    "Journal [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Authors: " ++ unwords (map show authors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (DVD id director actors title releaseDate duration copies) =
    "DVD [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Director: " ++ show director ++ "\n" ++
    "  Actors: " ++ unwords (map show actors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Duration: " ++ show duration ++ " minutes\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)
    
instance Show User where
  show (Student p borrowed) =
    "Student: " ++ show p ++ "\n" ++
    "  Borrowed: " ++ show (length borrowed) ++ " items"

  show (Professor p borrowed) =
    "Professor: " ++ show p ++ "\n" ++
    "  Borrowed: " ++ show (length borrowed) ++ " items"
instance Eq User where
  (Student p1 _) == (Student p2 _) = p1 == p2
  (Professor p1 _) == (Professor p2 _) = p1 == p2
  _ == _ = False
    
prettyCatalog :: Catalog -> String
prettyCatalog = unlines . map show


In [3]:
--Prueba
exampleDate :: Date
exampleDate = Date 18 "November" 2024

author1 :: Person
author1 = Person "John" "Doe" (Date 10 "January" 1980)

exampleBook :: Publication
exampleBook = Book 
  "B001" 
  author1 
  "Functional Programming in Haskell" 
  300 
  [Copy False Nothing Nothing Nothing]

exampleStudent :: User
exampleStudent = Student (Person "Alice" "Smith" (Date 5 "May" 2001)) []

exampleCatalog :: Catalog
exampleCatalog = [exampleBook]

exampleDate

author1

exampleBook

exampleStudent

exampleCatalog


18 November 2024

John Doe (10 January 1980)

Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available

Student: Alice Smith (5 May 2001)
  Borrowed: 0 items

[Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available
]

In [4]:
-- Ejemplo de fecha
exampleLoanDate :: Date
exampleLoanDate = Date 10 "November" 2024

exampleReturnDate :: Date
exampleReturnDate = Date 10 "December" 2024

-- Ejemplo de personas
student1 :: User
student1 = Student (Person "Alice" "Smith" (Date 5 "May" 2001)) [Copy True (Just student1) (Just exampleLoanDate) (Just exampleReturnDate)]

professor1 :: User
professor1 = Professor (Person "Dr." "Johnson" (Date 12 "August" 1975)) [Copy True (Just professor1) (Just exampleLoanDate) (Just exampleReturnDate)]

-- Ejemplo de copias de un libro
exampleCopies :: [Copy]
exampleCopies = 
  [ Copy True (Just student1) (Just exampleLoanDate) (Just exampleReturnDate)  -- Prestada a un estudiante
  , Copy True (Just professor1) (Just exampleLoanDate) (Just exampleReturnDate) -- Prestada a un profesor
  , Copy False Nothing Nothing Nothing  -- Disponible
  , Copy False Nothing Nothing Nothing  -- Disponible
  ]

-- Ejemplo de libro con varias copias
exampleBookWithCopies :: Publication
exampleBookWithCopies = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  exampleCopies

-- Catálogo con el libro
exampleCatalogWithCopies :: Catalog
exampleCatalogWithCopies = [exampleBookWithCopies]

exampleBookWithCopies

exampleCatalogWithCopies

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

In [5]:
-- Ejemplo de otro libro con copias
exampleCopies2 :: [Copy]
exampleCopies2 = 
  [ Copy True (Just student1) (Just (Date 1 "November" 2024)) (Just (Date 8 "November" 2024)) -- Prestada a un estudiante
  , Copy False Nothing Nothing Nothing  -- Disponible
  , Copy False Nothing Nothing Nothing  -- Disponible
  ]

exampleBookWithCopies2 :: Publication
exampleBookWithCopies2 = Book
  "B003"
  (Person "George" "Orwell" (Date 25 "June" 1903))
  "1984"
  328
  exampleCopies2

-- Catálogo actualizado con ambos libros
updatedCatalog :: Catalog
updatedCatalog = exampleCatalogWithCopies ++ [exampleBookWithCopies2]

-- Prueba de impresión del catálogo actualizado
prettyCatalog updatedCatalog


"Book [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 2/4 available\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\n"

## 3. An addToCatalog Publication Catalog function that adds a publication to the library catalog. If the publication is already in the catalog, its number of copies will be increased. 

In [6]:
addToCatalog :: Publication -> Catalog -> Catalog
addToCatalog pub [] = [pub]
addToCatalog pub (p:ps)
  | samePublication pub p = mergeCopies pub p : ps -- Si ya existe, combina las copias
  | otherwise             = p : addToCatalog pub ps -- Si no, sigue buscando
  where
    -- Verifica si dos publicaciones son la misma basándose en su ID único
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Combina las copias de dos publicaciones iguales
    mergeCopies :: Publication -> Publication -> Publication
    mergeCopies (Book id1 author1 title1 pages1 copies1) (Book _ _ _ _ copies2) =
      Book id1 author1 title1 pages1 (copies1 ++ copies2)
    mergeCopies (Journal id1 authors1 title1 release1 pages1 copies1) (Journal _ _ _ _ _ copies2) =
      Journal id1 authors1 title1 release1 pages1 (copies1 ++ copies2)
    mergeCopies (DVD id1 director1 actors1 title1 release1 duration1 copies1) (DVD _ _ _ _ _ _ copies2) =
      DVD id1 director1 actors1 title1 release1 duration1 (copies1 ++ copies2)
    mergeCopies pub1 _ = pub1 -- Por seguridad, devuelve pub1 en otros casos

-- Nueva copia para un libro existente
newCopiesForPrideAndPrejudice :: [Copy]
newCopiesForPrideAndPrejudice = 
  [ Copy False Nothing Nothing Nothing,  -- Nueva copia disponible
    Copy False Nothing Nothing Nothing   -- Otra nueva copia disponible
  ]

-- Publicación con nuevas copias de "Pride and Prejudice"
newPrideAndPrejudice :: Publication
newPrideAndPrejudice = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  newCopiesForPrideAndPrejudice

-- Nuevo libro completamente nuevo
newBook :: Publication
newBook = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  [Copy False Nothing Nothing Nothing]

-- Actualizar el catálogo
updatedCatalog2 :: Catalog
updatedCatalog2 = addToCatalog newPrideAndPrejudice $ addToCatalog newBook updatedCatalog

-- Prueba de impresión
prettyCatalog updatedCatalog2


"Book [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 4/6 available\nAvailable\nAvailable\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\nBook [B004]\n  Title: Harry Potter and the Philosopher's Stone\n  Author: J.K. Rowling (31 July 1965)\n  Pages: 223\n  Copies: 1/1 available\nAvailable\n\n"

In [7]:
-- Nuevas copias para "Harry Potter and the Philosopher's Stone"
newCopiesForHarryPotter :: [Copy]
newCopiesForHarryPotter = 
  [ Copy False Nothing Nothing Nothing,  -- Nueva copia disponible
    Copy False Nothing Nothing Nothing   -- Otra nueva copia disponible
  ]

-- Publicación con nuevas copias de "Harry Potter"
newBook2 :: Publication
newBook2 = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  newCopiesForHarryPotter

-- Actualizar el catálogo con newBook2 (se fusionan las copias)
finalCatalog :: Catalog
finalCatalog = addToCatalog newBook2 updatedCatalog2

-- Prueba de impresión
prettyCatalog finalCatalog


"Book [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 4/6 available\nAvailable\nAvailable\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n  Borrowed: 1 items | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\nBook [B004]\n  Title: Harry Potter and the Philosopher's Stone\n  Author: J.K. Rowling (31 July 1965)\n  Pages: 223\n  Copies: 3/3 available\nAvailable\nAvailable\nAvailable\n\n"

## 4. A removeFromCatalog Publication Catalog that removes a copy of the publication from the library catalog. If it is the last copy, the publication will be removed. If the catalog does not contain that publication an error will be raised.

In [8]:
removeFromCatalog :: Publication -> Catalog -> Either String Catalog
removeFromCatalog pub [] = Left "Error: The publication is not in the catalog."
removeFromCatalog pub (p:ps)
  | samePublication pub p =
      let updatedPub = removeCopy pub
      in if noCopiesLeft updatedPub
         then Right ps -- Elimina la publicación completamente
         else Right (updatedPub : ps) -- Actualiza la publicación
  | otherwise = fmap (p :) (removeFromCatalog pub ps) -- Sigue buscando
  where
    -- Verifica si dos publicaciones son iguales por su ID único
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Elimina una copia de la publicación (la primera disponible o prestada)
    removeCopy :: Publication -> Publication
    removeCopy (Book id1 author1 title1 pages1 copies1) =
      Book id1 author1 title1 pages1 (tail copies1)
    removeCopy (Journal id1 authors1 title1 release1 pages1 copies1) =
      Journal id1 authors1 title1 release1 pages1 (tail copies1)
    removeCopy (DVD id1 director1 actors1 title1 release1 duration1 copies1) =
      DVD id1 director1 actors1 title1 release1 duration1 (tail copies1)
    removeCopy pub = pub -- Por seguridad

    -- Verifica si no quedan copias en la publicación
    noCopiesLeft :: Publication -> Bool
    noCopiesLeft (Book _ _ _ _ copies1)    = null copies1
    noCopiesLeft (Journal _ _ _ _ _ copies1) = null copies1
    noCopiesLeft (DVD _ _ _ _ _ _ copies1) = null copies1


In [9]:
-- Catálogo inicial

catalogForRemoval :: Catalog
catalogForRemoval = finalCatalog
finalCatalog
exampleBookWithCopies
-- Eliminar una copia de "Pride and Prejudice"
result1 :: Either String Catalog
result1 = removeFromCatalog exampleBookWithCopies catalogForRemoval
result1
-- Eliminar "1984" completamente
{-result2 :: Either String Catalog
result2 = removeFromCatalog (Book "B003" undefined undefined undefined undefined) <$> result1
result2
-- Intentar eliminar una publicación inexistente
result3 :: Either String Catalog
result3 = removeFromCatalog (Book "B999" undefined undefined undefined undefined) <$> result2
-}
-- Prueba de impresión de resultados
printCatalogResult :: Either String Catalog -> IO ()
printCatalogResult (Left err) = putStrLn $ "Error: " ++ err
printCatalogResult (Right catalog) = putStrLn $ prettyCatalog catalog


   --result1
  --t (removeFromCatalog exampleBookWithCopies catalogForRemoval) 
  --printCatalogResult result1
  
  --printCatalogResult result3


[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 4/6 available
Available
Available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
,Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 1 November 2024 | Return Date: 8 November 2024
Available
Available
,Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
Available
Available
Available
]

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

Right [Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/3 available
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
,Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 1 November 2024 | Return Date: 8 November 2024
Available
Available
,Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
Available
Available
Available
]

In [10]:
exampleBookWithCopies
exampleCatalogWithCopies
removeFromCatalog exampleBookWithCopies exampleCatalogWithCopies

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

Right [Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/3 available
Borrowed by Professor: Dr. Johnson (12 August 1975)
  Borrowed: 1 items | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

## 5. A borrowedPublications User Catalog function that returns the publications the user has borrowed. They must be sort by closest return date, in case of tie in return date, it will first show books, then journals and finally DVDs. 

In [11]:
-- Función principal para obtener las publicaciones prestadas por un usuario
borrowedPublications :: User -> Catalog -> [Publication]
borrowedPublications user catalog = 
  let borrowedCopies = [ publication | publication <- catalog, 
                                     any (isBorrowedByUser user) (copies publication) ]
  in sortPublications borrowedCopies

-- Función auxiliar para verificar si una copia está prestada al usuario
isBorrowedByUser :: User -> Copy -> Bool
isBorrowedByUser user copy = 
  case borrower copy of
    Just u  -> u == user  -- Si el prestatario es el usuario, es una copia tomada por él.
    Nothing -> False      -- Si no está prestada, no la consideramos.

-- Función para ordenar las publicaciones
sortPublications :: [Publication] -> [Publication]
sortPublications [] = []
sortPublications (x:xs) = insertPublication x (sortPublications xs)

-- Función para insertar una publicación en el lugar adecuado
insertPublication :: Publication -> [Publication] -> [Publication]
insertPublication pub [] = [pub]
insertPublication pub (x:xs)
  | comparator pub x == LT = pub : x : xs  -- Si pub debe ir antes de x
  | otherwise = x : insertPublication pub xs  -- Sino, seguimos buscando

-- Comparador para ordenar publicaciones
comparator :: Publication -> Publication -> Ordering
comparator pub1 pub2 = 
  case compareReturnDate pub1 pub2 of
    EQ -> compareType pub1 pub2
    result -> result

-- Función para comparar las fechas de retorno
compareReturnDate :: Publication -> Publication -> Ordering
compareReturnDate pub1 pub2 =
  let returnDate1 = minimum (map returnDate (copies pub1))
      returnDate2 = minimum (map returnDate (copies pub2))
  in compare returnDate1 returnDate2

-- Función para comparar el tipo de publicación (libro primero, luego revista, luego DVD)
compareType :: Publication -> Publication -> Ordering
compareType (Book _ _ _ _ _) (Book _ _ _ _ _) = EQ
compareType (Book _ _ _ _ _) _ = LT
compareType _ (Book _ _ _ _ _) = GT
compareType (Journal _ _ _ _ _ _) (Journal _ _ _ _ _ _) = EQ
compareType (Journal _ _ _ _ _ _) _ = LT
compareType _ (Journal _ _ _ _ _ _) = GT
compareType (DVD _ _ _ _ _ _ _) (DVD _ _ _ _ _ _ _) = EQ
compareType (DVD _ _ _ _ _ _ _) _ = LT
compareType _ (DVD _ _ _ _ _ _ _) = GT

Line 17: Use foldr
Found:
sortPublications [] = []
sortPublications (x : xs)
  = insertPublication x (sortPublications xs)
Why not:
sortPublications xs = foldr insertPublication [] xsLine 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 44: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 45: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 47: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 48: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 50: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 51: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}

In [32]:

testUser = Student 
  (Person "John" "Doe" (Date 1 "January" 2000)) 
  [borrowedCopy1, borrowedCopy2, borrowedCopy3]


borrowedCopy1 = Copy True (Just testUser) (Just (Date 20 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy2 = Copy True (Just testUser) (Just (Date 15 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy3 = Copy True (Just testUser) (Just (Date 18 "November" 2024)) (Just (Date 28 "November" 2024))
borrowedCopy4 = Copy True (Just testUser) (Just (Date 24 "November" 2024)) (Just (Date 28 "November" 2024))

book1, book2, journal1 :: Publication
book1 = Book "B001" (Person "Author1" "Surname1" (Date 1 "January" 1970)) 
                   "Functional Programming in Haskell" 300 [borrowedCopy1]
book2 = Book "B002" (Person "Author2" "Surname2" (Date 1 "February" 1980)) 
                   "Learn You a Haskell" 250 [borrowedCopy2]
journal1 = Journal "J001" [Person "Author3" "Surname3" (Date 1 "March" 1990)] 
                         "Advanced Haskell Research" (Date 1 "March" 2024) 
                         50 [borrowedCopy3]
--dvd1 = DVD "D001" testUser [testUser] "Literature on Screen" (Date 1 "April" 2024) 120 []
dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 [borrowedCopy4]

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]

: 

In [13]:
testUser
print("-----------------------------------")
testCatalog
print("-----------------------------------")
borrowedPublications testUser testCatalog

Line 2: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"Line 4: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"

: 

## 6. A publicationsByAuthor Author Catalog that returns a list with all the publications for that author in the Catalog or the empty list if that person has not authored any publication. Create also equivalent booksByAuthor, journalsByAuthor and DVDsByAuthor that restrict the search to books, journals and DVDs respectively. Notice in DVD both directors and actors must be considered. 

In [14]:
-- General function to find all publications by a given author
publicationsByAuthor :: Person -> Catalog -> [Publication]
publicationsByAuthor author catalog =
  filter authoredBy catalog
  where
    -- Checks if a publication is authored by the given person
    authoredBy :: Publication -> Bool
    authoredBy (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredBy (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredBy (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    
booksByAuthor :: Person -> Catalog -> [Publication]
booksByAuthor author catalog =
  filter authoredByBook catalog
  where
    -- Checks if a book is authored by the given person
    authoredByBook :: Publication -> Bool
    authoredByBook (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredByBook _ = False

journalsByAuthor :: Person -> Catalog -> [Publication]
journalsByAuthor author catalog =
  filter authoredByJournal catalog
  where
    -- Checks if a journal is authored by the given person
    authoredByJournal :: Publication -> Bool
    authoredByJournal (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredByJournal _ = False

dvdsByAuthor :: Person -> Catalog -> [Publication]
dvdsByAuthor author catalog =
  filter authoredByDVD catalog
  where
    -- Checks if a DVD is directed or acted by the given person
    authoredByDVD :: Publication -> Bool
    authoredByDVD (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    authoredByDVD _ = False


Line 3: Eta reduce
Found:
publicationsByAuthor author catalog = filter authoredBy catalog
Why not:
publicationsByAuthor author = filter authoredByLine 13: Eta reduce
Found:
booksByAuthor author catalog = filter authoredByBook catalog
Why not:
booksByAuthor author = filter authoredByBookLine 22: Eta reduce
Found:
journalsByAuthor author catalog = filter authoredByJournal catalog
Why not:
journalsByAuthor author = filter authoredByJournalLine 31: Eta reduce
Found:
dvdsByAuthor author catalog = filter authoredByDVD catalog
Why not:
dvdsByAuthor author = filter authoredByDVD

In [15]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]


In [16]:
publicationsByAuthor author1 testCatalog
"------------------------------------"
booksByAuthor author1 testCatalog
"------------------------------------"
journalsByAuthor author1 testCatalog
"------------------------------------"
dvdsByAuthor author2 testCatalog


[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
,Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"------------------------------------"

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"------------------------------------"

[DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

## 7. Functions publicationsByTitle String Catalog and publicationsByDate Date Catalog that return all the publications with that title or release date. 

In [17]:
-- Function to find all publications with the given title
publicationsByTitle :: String -> Catalog -> [Publication]
publicationsByTitle title catalog =
  filter hasTitle catalog
  where
    -- Checks if a publication has the given title
    hasTitle :: Publication -> Bool
    hasTitle (Book _ _ pubTitle _ _) = pubTitle == title
    hasTitle (Journal _ _ pubTitle _ _ _) = pubTitle == title
    hasTitle (DVD _ _ _ pubTitle _ _ _) = pubTitle == title

-- Function to find all publications with the given release date
publicationsByDate :: Date -> Catalog -> [Publication]
publicationsByDate date catalog =
  filter hasReleaseDate catalog
  where
    -- Checks if a publication has the given release date
    hasReleaseDate :: Publication -> Bool
    hasReleaseDate (Book _ _ _ _ _) = False  -- Books don't have a specific release date in this case
    hasReleaseDate (Journal _ _ _ pubReleaseDate _ _) = pubReleaseDate == date
    hasReleaseDate (DVD _ _ _ _ pubReleaseDate _ _) = pubReleaseDate == date



Line 3: Eta reduce
Found:
publicationsByTitle title catalog = filter hasTitle catalog
Why not:
publicationsByTitle title = filter hasTitleLine 14: Eta reduce
Found:
publicationsByDate date catalog = filter hasReleaseDate catalog
Why not:
publicationsByDate date = filter hasReleaseDateLine 19: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

In [18]:
let catalog = [book1, journal1, dvd1] -- Replace with your actual catalog of publications

-- Find all publications with the title "Haskell Programming"
let publicationsWithTitle = publicationsByTitle "Pride and Prejudice" catalog
print publicationsWithTitle
"----------------------------------------------------------------------"
-- Find all publications released on a specific date (e.g., "1 Dec 2024")
let date = Date 1 "January" 2024
let publicationsWithDate = publicationsByDate date catalog
print publicationsWithDate


[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

## 8. A search Author Title Date function that can receive the author, the title and/or the date and returns all the publications matching the criteria. Notice that the function must work for any combination of the three items (for example searching just by author, or by author and date) and that some, or even the three of them, could be empty.

In [19]:
searchAuthorTitleDate :: Maybe Person -> Maybe String -> Maybe Date -> Catalog -> [Publication]
searchAuthorTitleDate author title date catalog = filter matchesCriteria catalog
  where
    -- Checks if a publication matches the given criteria
    matchesCriteria :: Publication -> Bool
    matchesCriteria pub =
      (authorMatches pub) && (titleMatches pub) && (dateMatches pub)

    -- Check if the publication matches the author criterion
    authorMatches :: Publication -> Bool
    authorMatches (Book _ bookAuthor _ _ _) = maybe True (== bookAuthor) author
    authorMatches (Journal _ journalAuthors _ _ _ _) = maybe True (`elem` journalAuthors) author
    authorMatches (DVD _ director actors _ _ _ _) = maybe True (== director) author || maybe False (`elem` actors) author

    -- Check if the publication matches the title criterion
    titleMatches :: Publication -> Bool
    titleMatches (Book _ _ bookTitle _ _) = maybe True (== bookTitle) title
    titleMatches (Journal _ _ journalTitle _ _ _) = maybe True (== journalTitle) title
    titleMatches (DVD _ _ _ dvdTitle _ _ _) = maybe True (== dvdTitle) title

    -- Check if the publication matches the date criterion
    dateMatches :: Publication -> Bool
    dateMatches (Book _ _ _ _ _) = True  -- Books are not filtered by date in this case
    dateMatches (Journal _ _ _ journalDate _ _) = maybe True (== journalDate) date
    dateMatches (DVD _ _ _ _ dvdDate _ _) = maybe True (== dvdDate) date


Line 2: Eta reduce
Found:
searchAuthorTitleDate author title date catalog
  = filter matchesCriteria catalog
Why not:
searchAuthorTitleDate author title date = filter matchesCriteriaLine 7: Redundant bracket
Found:
(authorMatches pub) && (titleMatches pub) && (dateMatches pub)
Why not:
authorMatches pub && (titleMatches pub) && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
titleMatches pub && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
(titleMatches pub) && dateMatches pubLine 23: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

In [20]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]
let catalog = [book1, journal1, dvd1] -- Replace with actual catalog

-- Search for publications by "John Doe" (author), with title "Haskell Programming"
let results = searchAuthorTitleDate (Just (Person "Jane" "Austen" (Date 16 "December" 1775))) (Just "Pride and Prejudice") Nothing catalog
print results
"----------------------------------------------------"
-- Search for publications with the title "Haskell Programming", regardless of author or date
let resultsByTitle = searchAuthorTitleDate Nothing (Just "Victorian Literature") Nothing catalog
print resultsByTitle
"----------------------------------------------------"
-- Search for publications by author "John Doe" released on a specific date
let resultsByAuthorAndDate = searchAuthorTitleDate (Just (Person "Charles" "Dickens" (Date 7 "February" 1812))) Nothing (Just (Date 1 "April" 2024)) catalog
print resultsByAuthorAndDate


[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"----------------------------------------------------"

[DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

## 9. A nextWorkingDate days Date [Date] function that receives a positive number of days, a Date and a list of holidays and returns the next working day (not in weekend or holiday) after those days. It will be used to calculate loan and return dates for publications, as the library is closed during weekends. Consider leap years and the fact that 1st January 2025 will be Wednesday. 

In [59]:
--nextWorkingDate :: Int -> Date -> [Date] -> Date
-- Eq and Ord instances for Date
instance Eq Date where
  (Date d1 m1 y1) == (Date d2 m2 y2) =
    y1 == y2 && monthIndex m1 == monthIndex m2 && d1 == d2

instance Ord Date where
  compare (Date d1 m1 y1) (Date d2 m2 y2) =
    compare (y1, monthIndex m1, d1) (y2, monthIndex m2, d2)
    
isLeapYear :: Int -> Bool
isLeapYear year = (year `mod` 4 == 0 && year `mod` 100 /= 0) || (year `mod` 400 == 0)

daysInMonth :: Int -> Int -> Int
daysInMonth month year
  | month == 2 = if isLeapYear year then 29 else 28
  | month `elem` [4, 6, 9, 11] = 30
  | otherwise = 31

isWeekend :: Date -> Bool
isWeekend (Date d m y) =
  let dayOfYear = cumulativeDays d m y
      weekday = (dayOfYear + daysTo2025 y) `mod` 7
  in weekday == 5 || weekday == 6 -- Saturday or Sunday
  where
    -- Days from start of year to this date
    cumulativeDays :: Int -> String -> Int -> Int
    cumulativeDays d m y = d + sum (map (\month -> daysInMonth month y) [1..monthIndex m - 1])
    
    -- Convert month name to an index
monthIndex :: String -> Int
monthIndex month = case month of
    "January" -> 1; "February" -> 2; "March" -> 3; "April" -> 4; "May" -> 5
    "June" -> 6; "July" -> 7; "August" -> 8; "September" -> 9; "October" -> 10
    "November" -> 11; "December" -> 12; _ -> error "Invalid month"

    -- Calculate the number of days from 2025 back to a given year
daysTo2025 :: Int -> Int
daysTo2025 y
    | y == 2025 = 0
    | y < 2025 = sum (map (\yr -> if isLeapYear yr then 366 else 365) [y..2024])
    | y > 2025 = negate $ sum (map (\yr -> if isLeapYear yr then 366 else 365) [2025..y - 1])

advanceDate :: Date -> Date
advanceDate (Date d m y)
  | d < daysInMonth (monthIndex m) y = Date (d + 1) m y
  | m == "December" = Date 1 "January" (y + 1)
  | otherwise = Date 1 (nextMonth m) y
  where
    nextMonth :: String -> String
    nextMonth month = case month of
      "January" -> "February"; "February" -> "March"; "March" -> "April"
      "April" -> "May"; "May" -> "June"; "June" -> "July"
      "July" -> "August"; "August" -> "September"; "September" -> "October"
      "October" -> "November"; "November" -> "December"; _ -> error "Invalid month"

nextWorkingDate :: Int -> Date -> [Date] -> Date
nextWorkingDate 0 date holidays = skipNonWorking date holidays -- Asegurarse de que el día final sea laboral
nextWorkingDate days date holidays =
  let nextDate = advanceDate date
  in if isWorkingDay nextDate holidays
       then nextWorkingDate (days - 1) nextDate holidays
       else nextWorkingDate days nextDate holidays
isWorkingDay :: Date -> [Date] -> Bool
isWorkingDay date holidays = not (isWeekend date) && date `notElem` holidays


-- Skip weekends and holidays
skipNonWorking :: Date -> [Date] -> Date
skipNonWorking date holidays
  | isWeekend date || date `elem` holidays = skipNonWorking (advanceDate date) holidays
  | otherwise = date


Line 28: Avoid lambda using `infix`
Found:
(\ month -> daysInMonth month y)
Why not:
(`daysInMonth` y)

In [58]:
exampleStartDate :: Date
exampleStartDate = Date 29 "December" 2024 -- Monday

holidays :: [Date]
holidays = [Date 1 "January" 2025, Date 6 "January" 2025] -- New Year's Day and Epiphany
--holidays = []

nextWorkingDate 0 exampleStartDate holidays
--putStrLn $ "Next working day: " ++ show result


29 December 2024

In [48]:
exampleStartDate `elem` holidays

False

In [186]:
-- Definición de Date
data Date = Date { day :: Int, month :: String, year :: Int }
  deriving (Show, Eq)

-- Helper: Calcular días en un mes
daysInMonth :: Int -> Int -> Int
daysInMonth 2 year = if isLeapYear year then 29 else 28
daysInMonth month _ | month `elem` [4, 6, 9, 11] = 30
daysInMonth _ _ = 31

-- Helper: Verificar si un año es bisiesto
isLeapYear :: Int -> Bool
isLeapYear year = (year `mod` 4 == 0 && year `mod` 100 /= 0) || (year `mod` 400 == 0)

-- Helper: Avanzar un día
advanceDate :: Date -> Date
advanceDate (Date d m y)
  | d < daysInMonth (monthIndex m) y = Date (d + 1) m y
  | m == "December" = Date 1 "January" (y + 1)
  | otherwise = Date 1 (nextMonth m) y
  where
    nextMonth :: String -> String
    nextMonth "January" = "February"
    nextMonth "February" = "March"
    nextMonth "March" = "April"
    nextMonth "April" = "May"
    nextMonth "May" = "June"
    nextMonth "June" = "July"
    nextMonth "July" = "August"
    nextMonth "August" = "September"
    nextMonth "September" = "October"
    nextMonth "October" = "November"
    nextMonth "November" = "December"
    nextMonth _ = error "Invalid month"

    monthIndex :: String -> Int
    monthIndex "January" = 1
    monthIndex "February" = 2
    monthIndex "March" = 3
    monthIndex "April" = 4
    monthIndex "May" = 5
    monthIndex "June" = 6
    monthIndex "July" = 7
    monthIndex "August" = 8
    monthIndex "September" = 9
    monthIndex "October" = 10
    monthIndex "November" = 11
    monthIndex "December" = 12
    monthIndex _ = error "Invalid month"

-- Verificar si un día es fin de semana
-- isWeekend :: Date -> Bool
-- isWeekend (Date d m y) =
--   let dayOfYear = cumulativeDays d m y
--       weekday = (dayOfYear + daysTo2025 y + 1) `mod` 7
--   in weekday == 5 || weekday == 6 -- Sábado o domingo
--   where
--     cumulativeDays d m y = d + sum (map (\month -> daysInMonth month y) [1..monthIndex m - 1])
--     daysTo2025 y
--       | y == 2025 = 0
--       | y < 2025 = sum [if isLeapYear yr then 366 else 365 | yr <- [y..2024]]
--       | y > 2025 = negate $ sum [if isLeapYear yr then 366 else 365 | yr <- [2025..y-1]]
daysFromYearOne :: Date -> Int
daysFromYearOne (Date d m y) =
  let daysThisYear = cumulativeDays d m y
      daysPreviousYears = sum [if isLeapYear yr then 366 else 365 | yr <- [1..y - 1]]
  in daysThisYear + daysPreviousYears
  where
    cumulativeDays :: Int -> String -> Int -> Int
    cumulativeDays d m y = d + sum (map (\month -> daysInMonth month y) [1..monthIndex m - 1])

isWeekend :: Date -> Bool
isWeekend date =
  let daysSinceYearOne = daysFromYearOne date
      weekday = (daysSinceYearOne + 6) `mod` 7 -- 1 Jan year 1 = Thursday = day 4
  in weekday == 5 || weekday == 6 -- Saturday or Sunday
daysFromYearOne (Date 1 "January" 2025) `mod` 7
isWeekend (Date 1 "January" 2025)
-- Verificar si un día es laboral
isWorkingDay :: Date -> [Date] -> Bool
isWorkingDay date holidays = not (isWeekend date) && date `notElem` holidays

-- Función principal para calcular el próximo día laboral
nextWorkingDate :: Int -> Date -> [Date] -> Date
nextWorkingDate 0 date holidays = skipNonWorking date holidays -- Asegurarse de que el día final sea laboral
nextWorkingDate days date holidays =
  let nextDate = advanceDate date
  in if isWorkingDay nextDate holidays
       then nextWorkingDate (days - 1) nextDate holidays
       else nextWorkingDate days nextDate holidays

-- Saltar a un día laboral válido
skipNonWorking :: Date -> [Date] -> Date
skipNonWorking date holidays
  | isWorkingDay date holidays = date
  | otherwise = skipNonWorking (advanceDate date) holidays


Line 70: Avoid lambda using `infix`
Found:
(\ month -> daysInMonth month y)
Why not:
(`daysInMonth` y)

3

False

In [192]:
exampleStartDate = Date 31 "December" 2025
holidays = [Date 1 "January" 2026, Date 10 "January" 2025, Date 13 "January" 2025]
result = nextWorkingDate 2 exampleStartDate holidays
result

Date {day = 5, month = "January", year = 2026}

In [ ]:
isWeekend (holidays !! 0)
--weekDay exampleStartDate

Line 1: Use head
Found:
holidays !! 0
Why not:
head holidays

False

In [189]:
-- daysFromYearOne :: Date -> Int
-- daysFromYearOne (Date d m y) =
--   let daysThisYear = cumulativeDays d m y
--       daysPreviousYears = sum [if isLeapYear yr then 366 else 365 | yr <- [1..y - 1]]
--   in daysThisYear + daysPreviousYears
--   where
--     cumulativeDays :: Int -> String -> Int -> Int
--     cumulativeDays d m y = d + sum (map (\month -> daysInMonth month y) [1..monthIndex m - 1])

-- isWeekend :: Date -> Bool
-- isWeekend date =
--   let daysSinceYearOne = daysFromYearOne date
--       weekday = (daysSinceYearOne + 3) `mod` 7 -- 1 Jan year 1 = Thursday = day 4
--   in weekday == 5 || weekday == 6 -- Saturday or Sunday

-- advanceDate :: Date -> Date
-- advanceDate (Date d m y)
--   | d < daysInMonth (monthIndex m) y = Date (d + 1) m y
--   | m == "December" = Date 1 "January" (y + 1)
--   | otherwise = Date 1 (nextMonth m) y
--   where
--     nextMonth :: String -> String
--     nextMonth month = case month of
--       "January" -> "February"; "February" -> "March"; "March" -> "April"
--       "April" -> "May"; "May" -> "June"; "June" -> "July"
--       "July" -> "August"; "August" -> "September"; "September" -> "October"
--       "October" -> "November"; "November" -> "December"; _ -> error "Invalid month"

-- isWorkingDay :: Date -> [Date] -> Bool
-- isWorkingDay date holidays = not (isWeekend date) && date `notElem` holidays

-- skipNonWorking :: Date -> [Date] -> Date
-- skipNonWorking date holidays
--   | isWorkingDay date holidays = date
--   | otherwise = skipNonWorking (advanceDate date) holidays

-- nextWorkingDate :: Int -> Date -> [Date] -> Date
-- nextWorkingDate 0 date holidays = skipNonWorking date holidays
-- nextWorkingDate days date holidays =
--   let nextDate = advanceDate date
--   in if isWorkingDay nextDate holidays
--        then nextWorkingDate (days - 1) nextDate holidays
--        else nextWorkingDate days nextDate holidays


In [193]:
exampleStartDate = Date 30 "December" 2024
holidays = [Date 1 "January" 2025, Date 6 "January" 2025, Date 29 "December" 2023]
nextWorkingDate 1 exampleStartDate holidays


Date {day = 31, month = "December", year = 2024}

In [ ]:
nextWorkingDate 10 (Date 28 "December" 2023) holidays


Date {day = 12, month = "January", year = 2024}